In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils.preprocess import *
from utils.model import *
import datetime
from tqdm import tqdm

In [2]:
train_data = read_data('train')
test_data = read_data('test')

train_y = read_target_train()

## Получение данных о продолжительности различных операций

In [3]:
def cat_preprocess(df, drop_cols, encode_col):
    df = df.drop(drop_cols, axis = 1)
    df = pd.get_dummies(df, columns=[encode_col])
    return df

In [4]:
def cat_feat_extr(df, cols, ves_col):
    for NPLV in tqdm(df.NPLV.unique()):
        try:
            for col in cols:
                if not df[ves_col][df.index[(df['NPLV'] == NPLV) & (df[col] > 0)].to_list()].empty: 
                    for i in range(len(df[ves_col][df.index[(df['NPLV'] == NPLV) & (df[col] > 0)].to_list()])):
                        df.at[df.index[(df['NPLV'] == NPLV) & (df[col] > 0)].to_list()[i], col] = \
                                df[ves_col][df.index[(df['NPLV'] == NPLV) & (df[col] > 0)].to_list()[i]]
        except:
            try:
                for col in cols:
                    if not df[ves_col][df.index[(df['NPLV'] == NPLV) & (df[col] > 0)].to_list()].empty: 
                        for i in range(len(df[ves_col][df.index[(df['NPLV'] == NPLV) & (df[col] > 0)].to_list()])):
                            df.at[df.index[(df['NPLV'] == NPLV) & (df[col] > 0)].to_list()[i], col] = \
                                    df[ves_col][df.index[(df['NPLV'] == NPLV) & (df[col] > 0)].to_list()[i]]
            except:
                try:
                    for col in cols:
                        if not df[ves_col][df.index[(df['NPLV'] == NPLV) & (df[col] > 0)].to_list()].empty: 
                            for i in range(len(df[ves_col][df.index[(df['NPLV'] == NPLV) & (df[col] > 0)].to_list()])):
                                df.at[df.index[(df['NPLV'] == NPLV) & (df[col] > 0)].to_list()[i], col] = \
                                        df[ves_col][df.index[(df['NPLV'] == NPLV) & (df[col] > 0)].to_list()[i]]
                except:
                    for col in cols:
                        if not df[ves_col][df.index[(df['NPLV'] == NPLV) & (df[col] > 0)].to_list()].empty: 
                            for i in range(len(df[ves_col][df.index[(df['NPLV'] == NPLV) & (df[col] > 0)].to_list()])):
                                df.at[df.index[(df['NPLV'] == NPLV) & (df[col] > 0)].to_list()[i], col] = \
                                        df[ves_col][df.index[(df['NPLV'] == NPLV) & (df[col] > 0)].to_list()[i]]
                    
    df = df.drop([ves_col], axis = 1).groupby('NPLV').agg('sum')
    
    return df

хронометраж - время начала и конца различных операций во время плавки

In [5]:
train_data['chronom']['VR_NACH'] = pd.to_datetime(train_data['chronom']['VR_NACH'])
train_data['chronom']['VR_KON'] = pd.to_datetime(train_data['chronom']['VR_KON'])
train_data['chronom']['duration'] = (train_data['chronom']['VR_KON'] - train_data['chronom']['VR_NACH']).dt.total_seconds()

test_data['chronom']['VR_NACH'] = pd.to_datetime(test_data['chronom']['VR_NACH'])
test_data['chronom']['VR_KON'] = pd.to_datetime(test_data['chronom']['VR_KON'])
test_data['chronom']['duration'] = (test_data['chronom']['VR_KON'] - test_data['chronom']['VR_NACH']).dt.total_seconds()

In [6]:
names_train_nop = train_data['chronom']['NOP'].value_counts()
names_train_nop_stay = set(names_train_nop[names_train_nop > 100].to_dict().keys())
names_train_nop = set(names_train_nop.to_dict().keys())

names_test_nop = set(test_data['chronom']['NOP'].value_counts().to_dict().keys())

name_to_stay = names_train_nop_stay.intersection(names_test_nop)

In [7]:
names_to_del_train = names_train_nop.difference(name_to_stay)
names_to_del_test = names_test_nop.difference(name_to_stay)

In [8]:
prep_chronom_train = cat_preprocess(train_data['chronom'], ['TYPE_OPER', 'VR_NACH', 'VR_KON', 'O2'], 'NOP')
prep_chronom_test = cat_preprocess(test_data['chronom'], ['TYPE_OPER', 'VR_NACH', 'VR_KON', 'O2'], 'NOP')

In [9]:
prep_chronom_train_cols = list(prep_chronom_train.columns.drop(['NPLV', 'duration']))
prep_chronom_test_cols = list(prep_chronom_test.columns.drop(['NPLV', 'duration']))

In [10]:
prep_chronom_train = prep_chronom_train.fillna(0)
prep_chronom_test = prep_chronom_test.fillna(0)

prep_chronom_train['duration'] = prep_chronom_train['duration'].astype('int64')
prep_chronom_test['duration'] = prep_chronom_test['duration'].astype('int64')

In [11]:
prep_chronom_train[prep_chronom_train_cols] = prep_chronom_train[prep_chronom_train_cols].astype('int64')
prep_chronom_test[prep_chronom_test_cols] = prep_chronom_test[prep_chronom_test_cols].astype('int64')

In [12]:
cat_chronom_train = cat_feat_extr(prep_chronom_train, prep_chronom_train_cols, 'duration')
cat_chronom_test = cat_feat_extr(prep_chronom_test, prep_chronom_test_cols, 'duration')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 780/780 [00:23<00:00, 33.83it/s]


In [13]:
for col in names_to_del_train:
    col_ = 'NOP_' + col
    del cat_chronom_train[col_]

for col in names_to_del_test:
    col_ = 'NOP_' + col
    del cat_chronom_test[col_]

In [14]:
cat_chronom_train.to_csv('data_chronom/chronom_duration_train.csv')
cat_chronom_test.to_csv('data_chronom/chronom_duration_test.csv')